In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import re

In [25]:
from serpapi import GoogleSearch

organic_results = {}
cities = ['Rennes','Brest','Quimper','Lorient','Vannes']

for city in cities:
    params = {
        "engine": "google",
        "q": "Photographe grossesse naissance à {}".format(city),
        "location": "France",
        "api_key": "b0ff40020e4091183fb0147327fb3aef2ecfd9e44a6c01bb79fd4f7073ea6ade",
        "num": 50,
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results[city] = results['organic_results']

In [26]:
df = pd.DataFrame(columns = ['position', 'title', 'link', 'displayed_link', 'snippet',
       'about_this_result', 'cached_page_link', 'related_results',
       'related_pages_link', 'date','city'])
for city in cities:
    df_temp = pd.DataFrame(organic_results[city])
    df_temp['city'] = city
    df = df.append(df_temp,ignore_index = True)

In [27]:
df['displayed_link'] = df.displayed_link.apply(lambda x: x[0:x.find(' ›')])

In [28]:
def get_email(link):
    try:
        req = requests.get(link)
        email = re.search(r'[\w.+-]+@[\w-]+\.[com|fr|net|org]+',str(req.content)).group()
    except:
        email = '_no email found'
    return email

def get_phone(link):
    try:
        req = requests.get(link)
        soup = BS(req.content, "html.parser")
        contents = "".join(soup.strings)
        phone = re.search('0[1-7][-\.\s]\d{2}[-\.\s]\d{2}[-\.\s]\d{2}[-\.\s]\d{2}|0[1-7]\d{8}',str(contents)).group()
    except:
        phone = '_no phone found'
    return phone

def get_contact_page(link):
    try:
        req = requests.get(link)
    except:
        return "_broken link"

    soup = BS(req.content, "html.parser")

    for tag in soup.find_all("a", href=True):
        if "contact" in str(tag.text).lower():
            return str(tag["href"])
    return "_no contact page found"

def is_valid(x):
    mother = x[0]
    child = x[1]
    if child[0:3]=="htt" or child[0:2]=="ww":
        return child
    else:
        if child[0]=="/":
            return mother+child
        else:
            return mother+"/"+child

In [29]:
df['email'] = df['displayed_link'].apply(lambda x: get_email(x))

In [30]:
df['phone'] = df['displayed_link'].apply(lambda x: get_phone(x))

c:\users\thomas\appdata\local\programs\python\python38\lib\site-packages\bs4\__init__.py:309: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [56]:
df['contact_page'] = df['displayed_link'].apply(lambda x: get_contact_page(x))

c:\users\thomas\appdata\local\programs\python\python38\lib\site-packages\bs4\__init__.py:309: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [57]:
df

,position,title,link,displayed_link,snippet,about_this_result,cached_page_link,related_results,related_pages_link,date,city,rich_snippet,missing,must_include,sitelinks,email,phone,contact_page
0,1,Les meilleurs photographes grossesse à Rennes ...,https://photopresta.fr/photographe-grossesse/r...,https://photopresta.fr,Les 25 meilleurs photographes naissance & gros...,{'source': {'description': 'photopresta.fr was...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,support@photopresta.com,01 76 54 91 83,/contact
1,2,Les meilleurs photographes naissance et grosse...,https://monbebebonheur.fr/photographes/rennes/,https://monbebebonheur.fr,Réserver une séance avec les meilleurs photogr...,{'source': {'description': 'monbebebonheur.fr ...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,_no email found,_no phone found,javascript:zE.activate();
2,3,Séance photo maternité à Rennes - Olivia Poirier,https://oliviapoirier.fr/seance-photo-maternit...,https://oliviapoirier.fr,"Une séance photo pour votre grossesse, la nais...",{'source': {'description': 'oliviapoirier.fr w...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,_no email found,_no phone found,https://oliviapoirier.fr/contact/
3,4,"Photographe Rennes - Famille, Femme enceinte &...",https://www.meggmphotographies-rennes.com/2-2.htm,https://www.meggmphotographies-rennes.com,Megg'M Photographies immortalise les plus beau...,{'source': {'description': 'meggmphotographies...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,_no email found,06 99 89 25 32,https://www.fotostudio.io/lead_forms/179
4,5,Naissance - Pauline.F Photography,https://www.paulinefphotography.com/blog/nouve...,https://www.paulinefphotography.com,Naissance. QUAND RESERVER LA SEANCE NOUVEAU-NE...,{'source': {'description': 'paulinefphotograph...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,paulinefphotography@gmail.com,06.48.62.77.03,#contact-form
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,45,Photographe femme enceinte Séance photo grosse...,http://www.imagezvous.com/photographe-femme-en...,http://www.imagezvous.com,"Portrait de femme enceinte, future maman, je p...",{'source': {'description': 'This result comes ...,http://webcache.googleusercontent.com/search?q...,NaN,NaN,NaN,Vannes,NaN,NaN,NaN,NaN,_no email found,02 96 74 11 10,contacter-isabelle-photographe-portraitiste-br...
236,46,Photographe bébé 12 mois - Séance photos à dom...,http://www.miliephotographedelinstant.com/phot...,http://www.miliephotographedelinstant.com,photographe bébé 12 mois - Séance photos en Br...,{'source': {'description': 'This result comes ...,http://webcache.googleusercontent.com/search?q...,NaN,NaN,NaN,Vannes,NaN,NaN,NaN,NaN,_no email found,06.79.60.95.23,_no contact page found
237,47,Dehennault Yann jusqu'à 90% - VANNES | Groupon,https://www.groupon.fr/deals/dehennault-yann,https://www.groupon.fr,Shooting photo formule grossesse et naissance ...,{'source': {'description': 'This result comes ...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Vannes,"{'top': {'detected_extensions': {'rating': 5, ...",NaN,NaN,NaN,_no email found,_no phone found,_no contact page found
238,48,Photographe portrait Finistere | Morbihan | Qu...,https://www.hieronimus-art.com/Photographe-por...,https://www.hieronimus-art.com,"Votre séance photo de portrait famille, grosse...",{'source': {'description': 'This result comes ...,https://webcache.googleusercontent.com/search?...,NaN,NaN,NaN,Vannes,NaN,NaN,NaN,NaN,_no email found,_no phone found,Contact.php


In [78]:
def is_valid(x):
    mother = x[0]
    child = x[1]
    if child[0:3]=="htt" or child[0:2]=="ww":
        return child
    else:
        if child[0]=="/":
            return mother+child
        else:
            return mother+"/"+child

In [81]:
df["contact_page"] = df[["displayed_link","contact_page"]].apply(is_valid, axis=1)

In [83]:
df["email_2"] = df["contact_page"].apply(lambda x: get_email(x))

In [85]:
df.loc[df.email == "_no email found"].loc[df.email_2 == "_no email found"].shape

(159, 19)

In [93]:
df.loc[(df.email == "_no email found") & (df.email_2 == "_no email found")].shape

(159, 19)

In [94]:
df.shape

(240, 19)

In [95]:
df["phone_2"] = df["contact_page"].apply(lambda x: get_phone(x))

c:\users\thomas\appdata\local\programs\python\python38\lib\site-packages\bs4\__init__.py:309: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [97]:
df.loc[(df.phone == "_no phone found") & (df.phone_2 == "_no phone found") & (df.email == "_no email found") & (df.email_2 == "_no email found")].shape

(137, 20)

In [102]:
df_2 = pd.read_excel("LeadsBebebonheur.xlsx")

In [104]:
df_2['email'] = df_2['displayed_link'].apply(lambda x: get_email(x))

In [105]:
df_2['phone'] = df_2['displayed_link'].apply(lambda x: get_phone(x))

In [106]:
df_2['contact_page'] = df_2['displayed_link'].apply(lambda x: get_contact_page(x))

In [107]:
df_2["contact_page"] = df_2[["displayed_link","contact_page"]].apply(is_valid, axis=1)

In [108]:
df_2["email_2"] = df_2["contact_page"].apply(lambda x: get_email(x))

In [109]:
df_2["phone_2"] = df_2["contact_page"].apply(lambda x: get_phone(x))

In [110]:
df_2.shape

(262, 12)

In [111]:
df_2.loc[(df_2.phone == "_no phone found") & (df_2.phone_2 == "_no phone found") & (df_2.email == "_no email found") & (df_2.email_2 == "_no email found")].shape

(58, 12)

In [112]:
df_2.to_csv("mbbleads.csv")